# Importando dados do INMET

In [2]:
import csv
import pandas as pd
import numpy as np
import datetime as dt
import glob

In [3]:
# Importando todos os CSVs 
anos = range(2000,2021)

# Coletando todos os nomes e caminhos dos arquivos em CSV das séries
csv_lista = []
for i in anos:
    csv_lista.append(glob.glob("C:/Users/MKM/OneDrive/Área de Trabalho/UFSCAR/TCC/dados/{}/*.csv".format(i)))
lista_csv = [item for sublist in csv_lista for item in sublist]


In [4]:
# coletando um unico csv para capturar os nomes de suas colunas
series_df_aleatorio = pd.read_csv(lista_csv[300],  encoding='latin-1', sep =';', header = 8, decimal = ',')
coluns_names = [var.replace(" ", "_").replace(",", "").replace(":", "").replace(".", "").replace("ÇÃO","cao").replace("°","").replace("²","").lower() for var in series_df_aleatorio.columns]


In [ ]:
# Criando uma lista com os dataframes de cada estação e ano
df_series = []
for u in range(len(lista_csv)):
    try:
        series_df = pd.read_csv(lista_csv[u],  encoding='latin-1', sep =';', header = 8, decimal = ',')
        series_df.columns = coluns_names
        series_df.drop(columns = ['hora_(utc)'])
        df_series.append(series_df)
    except:
        pass

In [ ]:
#Coletando as primeiras 8 linhas de cada arquivo csv, pois temos metadados da estação nestas linhas
df_meta_data = []
for y in range(len(lista_csv)):
    with open(lista_csv[y],'r', encoding='latin-1') as fd:
        reader=csv.reader(fd)
        rows = list(reader)
    # Para cada csv criamos um dataframe de das 8 primeiras linhas
    meta_data = []
    for j in range(0,8):
        meta_data.append(rows[j][0].split(';'))
    df_meta_data.append(pd.DataFrame(meta_data, columns=['metricas','resultados']))


In [ ]:
# Agora retiramos apenas os valores de cada variavel dos metadados e colocamos em uma lista
for p in range(len(df_meta_data)):
    df_meta_data[p] = df_meta_data[p].resultados
    
# Aqui estamos criando tres colunas em cada dataframe de dados de uma estação por cada ano com a sigla do seu estados_df estado, cidade e o id da estação
for e in range(len(df_series)):
    df_series[e]['estado'] = df_meta_data[e][1]
    df_series[e]['cidade'] = df_meta_data[e][2]
    df_series[e]['estacao_id'] = df_meta_data[e][3]

coluns_names = coluns_names + ['estado', 'cidade', 'estacao_id']
coluns_names.remove('hora_(utc)') 

In [ ]:

# Neste looping estamos coletando todas as siglas de estados que temos nos dataframes
estados = []
for q in range(len(df_meta_data)):
    estados.append(df_meta_data[q][1])
estados = list(set(estados))
estados.sort()


# Mas neste outro processo estamos coletando qual é a sigla do estado e o index dela na lista de dataframes
estados_df = pd.DataFrame(index=np.arange(len(df_series)) , columns = ['estados', 'indice', 'id_estacao'])
for w in range(len(df_series)):
    estados_df['estados'].iloc[w] = df_series[w]['estado'].iloc[0]
    estados_df['indice'].iloc[w] = w
    estados_df['id_estacao'].iloc[w] = df_series[w]['estacao_id'].iloc[0]


In [ ]:
# Criando uma lista de dataframes por estado, onde todos os dados de todas as estações por estado se encontram em cada casa da lista
df_series_state = []
for x in range(len(estados)):
    df_series_state.append(pd.concat([df_series[l] for l in estados_df[estados_df.estados == estados[x]].indice.to_list()], names =coluns_names ))
    
# Transformando as data de cada dataframe em datetime. Além de criar para cada estado um dataframe com a média de temperatura entre as suas respectivas estações por dia 
for k in range(len(df_series_state)):
    df_series_state[k]['data_(yyyy-mm-dd)'] = df_series_state[k]['data_(yyyy-mm-dd)'].replace(regex='/', value = '-')
    df_series_state[k]['data_(yyyy-mm-dd)'] = pd.to_datetime(df_series_state[k]['data_(yyyy-mm-dd)'], format = '%Y-%m-%d')
    df_series_state[k] = df_series_state[k].replace(-9999, np.nan)
    df_series_state[k] = df_series_state[k].groupby('data_(yyyy-mm-dd)').mean().sort_values(by = ['data_(yyyy-mm-dd)'], ascending=True)
    df_series_state[k] = df_series_state[k].drop(columns = ['unnamed_19'])


In [ ]:
for t in range(len(df_series_state)):
    df_series_state[t].to_csv('C:/Users/MKM/OneDrive/Área de Trabalho/EAD 2021/TCC/dados_agregados/inmet_{}.csv'.format(estados[t]), index=True)